In [1]:
#to identify the medications provided to the patient
import spacy
med7 = spacy.load('en_core_med7_lg')
# create distinct colours for labels
col_dict = {}
seven_colours = ['#CEF6F5 ', '#5DADE2', '#a6e7ff', '#82E0AA', '#CCCCFF  ', '#45B39D', '#D289E3 ']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

In [2]:
#summarization module for sanitized document changes are reflected in RECORD;ADMISSION DATE;DISCHARGE DATE;DICTATED BY,ATTENDING
def san_summarization(filename):
    def listToString(s):  
                                                        # initialize an empty string 
        str1 = ""                                       # traverse in the string   
        for ele in s:  
            str1 += ele                                 # return string   
        return str1
    
    def content(pattern):
                                                        #pattern='PRINCIPAL DIAGNOSIS:(.*?)(:)'
        temp_var = re.search(pattern,text).group(0)
                                                        #Identifying the last label
        result = re.search('[ a-zA-Z]+\:$',temp_var).group(0)
                                                        #splitting each sentence based on full stop
        result_1=temp_var.split(".")
                                                        #creating a list having only principal diagnosis
        text_list=[]
        for i in result_1:
            if i != result:
                text_list.append(i)
                                                        #text_list
        final_result = '.'.join(text_list)
        final= final_result+'.'
        return final
    
    text=''
    with open(filename,"r") as a_file:
        for line in a_file:
            stripped_line = line.replace("\n"," ")
            text+=stripped_line
    
    
    import re
    p_d=""
    h_p_i=""
    p_m_h=""
    m_o_a=""
    allergies=""
    p_e_a=""
    if 'RECORD' in text:
        r=text.split()
        text=text.replace(r[1],'#####')
    if 'Admission Date:' in text:    
        a_d = re.findall(r"Admission Date: \[\d{4}]", text)
    if 'Discharge Date:' in text:    
        d_d = re.findall(r"Discharge Date: \[\d{4}]", text)    
    if 'PRINCIPAL DIAGNOSIS ON DISCHARGE:' in text:
        p_d = 'PRINCIPAL DIAGNOSIS ON DISCHARGE:(.*?)([a-bA-Z]:)'
        p_d = content(p_d)
    if 'HISTORY OF PRESENT ILLNESS:' in text:
        h_p_i='HISTORY OF PRESENT ILLNESS:(.*?)([a-bA-Z]:)'
        h_p_i=content(h_p_i)
    if 'PAST MEDICAL HISTORY:' in text:
        p_m_h='PAST MEDICAL HISTORY:(.*?)([a-bA-Z]:)'
        p_m_h=content(p_m_h)
    if 'MEDICATIONS ON ADMISSION:' in text:
        m_o_a='MEDICATIONS ON ADMISSION:(.*?)([a-bA-Z]:)'
        m_o_a=content(m_o_a)
    if 'PREOPERATIVE MEDICATIONS:' in text:
        p_m = 'PREOPERATIVE MEDICATIONS:(.*?)([a-bA-Z]:)'
        p_m = content(p_m)
    if 'ALLERGIES' in text:
        allergies='ALLERGIES:(.*?)([a-bA-Z]:)'
        allergies=content(allergies)
    if 'PHYSICAL EXAMINATION ON ADMISSION:' in text:
        p_e_a='PHYSICAL EXAMINATION ON ADMISSION:(.*?)([a-bA-Z]:)'
        p_e_a=content(p_e_a)
    if 'Attending:'in text:
        a=re.findall(r"Attending: \[PERSONS]",text)
    if 'Dictated By:' in text:
        d=re.findall(r"Dictated By: \[DOCTORS]",text)
    
    #appending the strings to make a new file
    res = r[0]+" "+r[1]+"\n"+listToString(a_d)+"\n"+listToString(d_d)+"\n"+p_d+"\n"+h_p_i+"\n"+p_m_h+"\n"+m_o_a+"\n"+p_m+"\n"+allergies+"\n"+p_e_a+"\n"+listToString(a[0])+"\n"+listToString(d)+"\n"
    res1 = res.split('\n')
    print(res1)
    with open('summerization.txt',"w") as f:
        for lis in res1:
            if lis != " ":
                f.write('%s\n' %lis)
    return res1

In [3]:
#calling the summarization function for the text document op.txt
summarized_text=san_summarization(r"C:\Users\Kavya\Downloads\op.txt")
#removal of blank spaces
while("" in summarized_text) : 
    summarized_text.remove("")
joined_string = "\n".join(summarized_text)
#load the entire extracted document to med7 model
d1=med7(joined_string)
spacy.displacy.render(d1, style='ent', page=True,jupyter = True,options=options)

['RECORD #####', 'Admission Date: [2005]', 'Discharge Date: [2005]', '', 'HISTORY OF PRESENT ILLNESS: [PERSON] is a [65.5] years old male with a history of rheumatic fever in the childhood who was found to have aortic stenosis about [5.5] years ago. He has been followed with serial echos. His most recent study shows critical AS. He admits to a few episodes of throat tightness with activity and a single episode of lightheadedness with vigorous exercise.', '', '', 'PREOPERATIVE MEDICATIONS: Lisinopril 5 mg daily , aspirin 81 mg daily , atorvastatin 80 mg daily , and metformin 850 mg b.i.d.', 'ALLERGIES: Topical reactions to unknown substance not latex. No known drug allergies.', '', 'Attending: [PERSONS]', 'Dictated By: [DOCTORS]', '']


In [4]:
#converting the obtained doc into pdf
import pdfkit
#optional configuration
config = pdfkit.configuration(wkhtmltopdf =r'C:/Users/Kavya/Desktop/hi/wkhtmltopdf/bin/wkhtmltopdf.exe')
html =  spacy.displacy.render(d1, style='ent', page=True,jupyter = False,options=options)
#change the path as per your convenience
with open("C:/Users/Kavya/Desktop/summohi.html", 'w+', encoding="utf-8") as fp:
    fp.write(html)

pdfkit.from_file('C:/Users/Kavya/Desktop/summohi.html','summohi.pdf',configuration=config )
fp.close()

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
